## Imports

In [ ]:
%run base.ipynb
%load_ext autoreload
%autoreload 2

import pickle
from pprint import pp as pprint
import tqdm.notebook as tqdm

import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import tabulate

# import signature q-function and q-learning algorithm
from sigqlearning_methods import train, test_multiple_episodes, test_single_episode
from sigqlearning_qfunctions import SigQFunction
import utils

If results and plots from this notebook should be saved set `save=True` and select an identifiying prefix used for file names when saving results and plots, e.g. current date with a letter or number.

In [ ]:
save_flag = False
file_prefix = '20240905_A'

# check if files with selected prefix exist
from pathlib import Path
if len(list(Path('../results').glob(file_prefix + '*'))) + \
    len(list(Path('../figures').glob(file_prefix + '*'))) != 0:
    file_prefix = input('Files with the chosen prefix already exist.\nPlease enter new prefix.')
print('Chosen prefix:', file_prefix)

## Environment 

In [ ]:
steps = 200
env = gym.make('MountainCar-v0', max_episode_steps=steps)

## Train Q-functions

We create the environment and specify the configuration of the Signature-Q-Function approximation and the training algorithm hyper-parameters.

In [ ]:
# signature config
sigq_params = dict(
    in_channels = 2, # time is added manually, no actions
    out_dimension = 3,
    sig_depth = 5,
    initial_basepoint = [-0.65, 1.], 
    initial_bias = 0.1
)

# training config
training_params = dict(
    episodes = 4000,
    discount=0.99,
    learning_rate = 0.05,
    learning_rate_decay = dict(mode='exponential', factor=0.995, end_value=1e-05),
        #dict(mode=None),
        #dict(mode='linear', end_value=1e-05, epochs=4000),
    epsilon = 0.3,
    epsilon_decay = dict(mode='linear', end_value=0.05, epochs=3000), 
        #dict(mode=None),
    decay_increment = 'success',
    batch_size=1,
)

Perform training runs.

In [ ]:
training_runs = 10

training_results = []
final_Q_functions = []

# run training algorithm loop
pbar = tqdm.trange(training_runs)
for run in pbar:
    pbar.set_description('Training runs')
    sigQfunction = SigQFunction(**sigq_params)
    results = train(env, sigQfunction, **training_params) 
    training_results.append(results)
    final_Q_functions.append(sigQfunction.state_dict())

### Plot training results

#### Single run
We plot the results for a specific run, set by ``run_id``. The results of interest are the reward obtained, the total loss occured, the car's end position, and the steps before reaching the goal or terminating, in each episode.

In [ ]:
run_id = -1
print('Number of successes in training run {}: {}'.format(
    run_id,sum(np.array(training_results[run_id][2])>=0.5)
))
utils.plot_results(training_results, run=run_id, ma_window=100,
                   title="Results for training run {}".format(run_id))

#### Averaged over all runs

We average the results over all training runs and calculate the mean and standard deviation of
- reward per episode
- loss per episode
- end position per episode
- steps until termination per episode

and plot the average results. Additionally we create a plot for each of the presented statistics and save them in ``../figures``.

In [ ]:
# select reward, loss, end position, steps for array
results_array = np.array(
    [ training_results[run][0:4] for run in range(len(training_results)) ], ndmin=3
)
training_results_means = results_array.mean(axis=0)
training_results_stds = results_array.std(axis=0)

utils.plot_mean_results(training_results_means, training_results_stds, 
                        title='Results averaged over training runs')
if save_flag:
    utils.save_mean_results_plots(training_results_means, training_results_stds,
                                  file_path='../figures/', file_id=file_prefix, show=False)

#### First Q-values

We check whether the Q-values at the beginning of the episode for converge towards the same value as the overall reward obtained over the episode. This is an indication, that the algorithm learns the true optimal Q-values of the problem. Unfortunately this is not the case here.

The first Q-values are plotted in two ways:
- The actual first Q-values saved during training for each episode and calculated for the respective observation $o_0$ encountered in each episode.
- The first Q-values for a fixed observation at $t=0$, for example $o_0 = (-0.5, 1.)$, calculated with the intermediate Q-functions saved during training each 10 episodes.

In [ ]:
# select run and if plot should be saved
run_id = -1
sigqfunction = SigQFunction(**sigq_params)

# plot and save
utils.plot_first_Q_values(training_results, run_id, step=10, 
                          window=(0,training_params['episodes']),
                          show=True, save=save_flag, 
                          file_path='../figures/{}_first_Q_values.png'.format(file_prefix))

start_pos = -0.4
utils.plot_first_Q_values(training_results, run_id, intermediate_qfunctions=True,
                          window=(1000,training_params['episodes']), time='down',
                          sigq_container=sigqfunction, start_position=start_pos, show=True)

#### First observation value
Additionally we take a look at the value of the history at time $t=0$ for the start position $p_0 = -0.5$, which corresponds to the middle of the interval $[-0.6, -0.4]$ in which the car is placed randomly at the start of the episode. The value is calculated as the Q-values averaged over actions and is given by
$$
    V_0(\hat{h}_0) = \frac{1}{3}\sum_{i=0}^2 Q(\hat{h}_0, a_i),   
$$
It gives the value over an episode following a greedy policy based on the current approximate Q-function $Q$. If $V_0(\hat{h}_0)$ converges towards the average reward over an episode, this indicates that the algorithm has converged to the true Q-values (at least for the Q-values at the start of the episode). Unfortunatly this is not the case here, since the algorithm over estimates Q-values.

The plot displays the mean and standard deviation of $V_0(\hat{h}_0)$ calculated over all performed training runs. 

In [ ]:
# create array containing saved values for all run
first_observations_array = np.array(
    [training_results[run][-3] for run in range(len(training_results))]
)
value_means = first_observations_array.mean(axis=0)
values_stds = first_observations_array.std(axis=0)

# plot and save
utils.plot_first_obs_value(value_means, values_stds, save=save_flag, 
                           file_path='../figures/{}first_values.png'.format(file_prefix))

## Test Q-functions

#### Single episode

We test the learned Q-approximation for a single episode and plot the observation history. The final Q-approximation, or some intermediate approximation saved every 10 steps during training, may be used.

In [ ]:
run_id = -1
intermediate_sigq_id = -1
sigqfunction = SigQFunction(**sigq_params)
sigqfunction.load_state_dict(training_results[run_id][-1][intermediate_sigq_id])

# single test episode
reward, success, steps, start, first_obs_value, history = test_single_episode(
    env, sigqfunction, epsilon=0.0, render=False
)
print(f"Success: {success}")
plt.plot(history)
plt.show()

#### Multiple episodes

We test the final Q-approximations from each training run over a number of episodes and report statistics. 

Checkpoints of the `signatureQFunction` approximations during training was saved every 10 episodes and after the last training episode in each training run. The final learned approximation corresponds to the last checkpoint, given as `state_dict` of the `SigQFunction` class instance at the checkpoint time. Other approximation from training may be tested by selecting the appropriate checkpoint.

In [ ]:
# choose number of test episodes and Q-function checkpoint
test_episodes = 500
sigq_checkpoint_id = -1

sigqfunction = SigQFunction(**sigq_params)
test_results = []

pbar = tqdm.trange(training_runs)
for run in pbar:
    pbar.set_description("Test runs".format(run))
    # load last Sig-Q approximation
    sigqfunction.load_state_dict(training_results[run][-1][sigq_checkpoint_id])
    results = test_multiple_episodes(env, sigqfunction, 
                                     test_episodes, epsilon=0.0)
    test_results.append(results)

### Test results
#### Statistics

We calculate the following statistics for each test run:
- Mean reward and standard deviation
- Number of successes 
- Mean episode length and standard deviation
- Minimum / maximum episode steps
- Mean starting position
- Mean first observation value

Additionally, we plot the episode length vs. starting position for a specified run, to gain a more qualitative insight into the solution that was learned by this training run.

In [ ]:
test_stats = []
cols = ['Mean\nreward', 'Std\nreward', 'Successes', 'Mean\nsteps', 'Std\nsteps', 'Min\nsteps', 'Max\nsteps', 'Mean first\nobs value']
rows = ['Run {}'.format(i) for i in range(len(test_results))]

for test_run in test_results:
    run_array = np.array(test_run[0:5])
    run_stats = []
    run_stats.append(run_array[0].mean()) # mean reward
    run_stats.append(run_array[0].std()) # std reward
    run_stats.append(int(run_array[1].sum())) # number successes
    run_stats.append(run_array[2].mean()) # mean episode length
    run_stats.append(run_array[2].std()) # std episode length
    run_stats.append(int(run_array[2].min())) # min / max episode steps
    run_stats.append(int(run_array[2].max())) # max episode steps
    run_stats.append(run_array[4].mean()) # first observation value
    
    test_stats.append(run_stats)

print(tabulate.tabulate(test_stats, headers=cols, showindex=rows, floatfmt='.4f'))

#### Confidence intervals
We report confidence intervals for rewards per test run.

In [ ]:
# confidence intervals
reward_conf_int = []
reward_means = []
mean_std_error = 0

for result in test_results:
    mean = np.mean(result[0])
    standard_error = stats.sem(result[0])
    confidence_interval = stats.norm.interval(0.95, loc=mean, scale=stats.sem(result[0]))
    reward_conf_int.append(confidence_interval)
    reward_means.append(mean)
    mean_std_error += standard_error/len(test_results)

print(f'\nMean std error of mean reward: {mean_std_error:.4f}')

# plot confidence intervals
fig_id = '20240905_A'
for (lower, upper), mean, y in zip(reward_conf_int, reward_means, range(training_runs)):
    plt.plot((lower, upper),(y,y),'b|-')
    plt.plot(mean, y, 'bo')
plt.yticks(range(training_runs), [f'Run {run+1}' for run in range(training_runs)], fontsize=11)
plt.xticks(fontsize=11) 
plt.figsize=(5.5, 4.125)
plt.tight_layout()
plt.savefig(f'../figures/{fig_id}_reward_confidence_intervals.png')
plt.show()

#### Scatterplots

In [ ]:
# scatterplots
run_id = -1

# start position vs. number of steps
plt.figure(figsize=(5.5, 4.125))        
plt.scatter(test_results[run_id][3], test_results[run_id][2], marker='x')
plt.xlabel("Start position", fontsize=11)
plt.ylabel("Number of steps", fontsize=11)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.savefig('../figures/{}_start_pos_vs_steps.png'.format(file_prefix))
plt.show()

# start position vs. reward
plt.figure(figsize=(5.5, 4.125))        
plt.scatter(test_results[run_id][3], test_results[run_id][0], marker='x')
plt.xlabel("Start position", fontsize=11)
plt.ylabel("Reward", fontsize=11)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.savefig('../figures/{}_start_pos_vs_reward.png'.format(file_prefix))
plt.show()

In [ ]:
run_id = -1

# number of steps vs. reward
plt.scatter(test_results[run_id][2], test_results[run_id][0], marker='x')
plt.xlabel("Number of steps", fontsize=11)
plt.ylabel("Reward", fontsize=11)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

# start position vs. end position
end_pos = [test_results[run_id][5][i][-1,0] for i in range(len(test_results[run_id][5]))]
plt.scatter(test_results[run_id][3], end_pos, marker='x')
plt.xlabel("Start position", fontsize=11)
plt.ylabel("End position", fontsize=11)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.show()

## Save results

In [ ]:
if save_flag:
    # create file name and write results
    file_path = '../results/' + file_prefix + '_fixed_setup_results.pkl'
    results_dict = dict(sigq_params=sigq_params, 
                        training_params=training_params, 
                        training_results=training_results,
                        test_results=test_results)
    with open(file_path, 'wb') as f:
        pickle.dump(results_dict, f) # serialize the list
    print('Results saved under \"{}\"'.format(file_path))

## Load results

Instead of performing new training and testing runs, prior saved training and testing results, together with the parameter configuration used in training, may be loaded. Saved results can be found in `../results/` and are identified by a distinct `file_prefix`, e.g. the date the runs were performed, in the format `YYYYMMDD`, together with an upper case letter which enumerates the result saved on the same date. To load results select the respective `file_prefix` of the results to be loaded and set `execute_cell_flag = True`.

The following objects are loaded:
- `training_runs`- int, number of performed training and test runs 
- `sigq_params` - dict containing the signature-Q-function parameters
- `training_params` - dict containing the training algorithm parameters
- `training_results` - list, entries are lists with training results of each run
- `test_results` - list,  entries are lists with test results of each run

**Note:** To display results after loading them, the respective cells above this section need to be executed. 

In [ ]:
execute_cell_flag = False

# set the file_prefix of the results to be loaded
load_file_prefix = '20240905_A'

if execute_cell_flag:
    file_path = '../results/' + load_file_prefix + '_fixed_setup_results.pkl'
    with open(file_path, 'rb') as f:
        results_dict = pickle.load(f)
    
    sigq_params = results_dict["sigq_params"]
    training_params = results_dict['training_params']
    training_results = results_dict["training_results"]
    test_results = results_dict["test_results"]
    training_runs = len(results_dict["training_results"])
    #comment_dict=results_dict['comment_dict']
    print('Training results from \"{}\" loaded.\n'.format(file_path))
    print('Training runs: {}.\nWith parameters:\n'.format(training_runs))
    pprint({key:results_dict[key] for key in results_dict if key not in ('training_results', 'test_results')})